In [1]:
from pyquil import get_qc, Program
from pyquil.gates import CNOT, Z, H,S, MEASURE, I
from pyquil.api import local_forest_runtime
from pyquil.quilbase import Declare
from pyquil.simulation.tools import lifted_gate, program_unitary
from pyquil.quil import *

In [2]:
import numpy as np
import random
from AverageFidelity import averageOfFidelity

In [3]:
num_qubits = 2

In [4]:
#The glossary of Clifford gates
clifford_glossary = []
clifford_glossary.extend([CNOT(i,j) for i in range(num_qubits) for j in range(num_qubits)])
for i in range(num_qubits): clifford_glossary.remove(CNOT(i,i))
clifford_glossary.extend([H(i) for i in range(num_qubits)])
clifford_glossary.extend([S(i) for i in range(num_qubits)])

In [5]:
len(clifford_glossary)

6

In [6]:
#First step choose m and the K_m sequences of Clifford group
m = 10
k_m = 10 #n. of diff sequences
n_m = 10  #n. of samples from a certain sequence
response_matrix = np.zeros((k_m,n_m))

for i_sequ in range(k_m):
    c_jm =  random.choices(clifford_glossary, k = m)
    #compute the unitary of circuit U
    prog = Program() #All qubits begin with |0> state
    for gate in c_jm:
        prog += gate

    c_jm_unitary = program_unitary(prog, n_qubits= num_qubits)

    #report one random adapted stabilizer
    s_i = np.identity(2**num_qubits)
    for q_num in range(num_qubits):
        s_i = np.matmul( lifted_gate( np.random.choice([I(q_num), Z(q_num)]), n_qubits = num_qubits ), s_i )
    s_i = np.matmul(np.matmul(c_jm_unitary, s_i), np.conj(c_jm_unitary) )
    stab_i_definition = DefGate("STAB_i", s_i)
    STAB_i = stab_i_definition.get_constructor() # Get the gate constructor

    n_tuple = tuple(range(num_qubits))
    prog += Program( stab_i_definition, STAB_i(*n_tuple) )

    #Measurments
    ro = prog.declare('ro', 'BIT', num_qubits)
    for q in range(num_qubits):
        prog += MEASURE(q, ro[q])

    prog = prog.wrap_in_numshots_loop(n_m)
    
    #Run the program
    qc = get_qc( str(num_qubits) + 'q-qvm')  # You can make any 'nq-qvm'
    executable = qc.compile(prog)
    result = qc.run(executable)
    measured_outcome = result.readout_data.get('ro')
    
    response_matrix[i_sequ,:] = 1 - np.bool_(np.sum(measured_outcome, axis = 1)) # 1 if it is equal to n_zero state
    

    
    


In [7]:
print(prog)

DEFGATE STAB_i:
    0.4999999999999998-1.1185571585378688e-17i, -1.1185571585378688e-17-0.4999999999999997i, -1.1185571585378688e-17+0.4999999999999998i, 0.4999999999999998+1.1185571585378686e-17i
    1.1185571585378688e-17+0.4999999999999998i, -0.4999999999999998+1.1185571585378686e-17i, -0.4999999999999998-1.1185571585378688e-17i, 1.1185571585378688e-17-0.4999999999999997i
    0.4999999999999998+1.1185571585378688e-17i, -1.1185571585378688e-17+0.4999999999999997i, -1.1185571585378688e-17-0.4999999999999998i, 0.4999999999999998-1.1185571585378686e-17i
    -1.1185571585378688e-17+0.4999999999999998i, 0.4999999999999998+1.1185571585378686e-17i, 0.4999999999999998-1.1185571585378688e-17i, -1.1185571585378688e-17-0.4999999999999997i

H 1
H 1
CNOT 1 0
CNOT 0 1
S 0
S 0
S 0
H 1
CNOT 0 1
CNOT 0 1
STAB_i 0 1
DECLARE ro BIT[2]
MEASURE 0 ro[0]
MEASURE 1 ro[1]



In [8]:
c_jm_unitary

array([[ 7.07106781e-01+0.00000000e+00j, -1.58187870e-17+0.00000000e+00j,
        -1.58187870e-17+0.00000000e+00j,  7.07106781e-01+0.00000000e+00j],
       [ 0.00000000e+00+1.58187870e-17j,  0.00000000e+00-7.07106781e-01j,
         0.00000000e+00-7.07106781e-01j,  0.00000000e+00+1.58187870e-17j],
       [ 7.07106781e-01+0.00000000e+00j,  1.58187870e-17+0.00000000e+00j,
        -1.58187870e-17+0.00000000e+00j, -7.07106781e-01+0.00000000e+00j],
       [ 0.00000000e+00+1.58187870e-17j,  0.00000000e+00+7.07106781e-01j,
         0.00000000e+00-7.07106781e-01j,  0.00000000e+00-1.58187870e-17j]])

In [9]:
response_matrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 1., 0., 0., 1., 1., 0.],
       [1., 1., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 1., 0., 1., 0., 1.],
       [0., 1., 0., 1., 1., 0., 1., 1., 0., 1.],
       [1., 1., 1., 0., 1., 0., 0., 0., 0., 0.]])

In [10]:
averageOfFidelity(response_matrix, k_m, n_m)

C:\Users\mohsen\Documents\Python_pros\git_controlled\Randomized_Benchmarking_protocol\AverageFidelity.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  R_Array = np.array( [int(1/np.log(1/p_jm)+0.5) for p_jm in Pacc_Array] ) #nearest integer is required


OverflowError: cannot convert float infinity to integer

In [ ]:
result